## Setup

In [1]:
import os, sys
sys.path.append(os.path.abspath('../../src/'))

import torch
import torch.optim as optim
import torch.nn as nn
import itertools
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from model.spectogram_dataset import SpectrogramDataset

import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mldogs (mldogs-wut). Use `wandb login --relogin` to force relogin


True

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


## CNN

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, dropout_conv_p=0.0, dropout_fc_p=0.0):
        super(Net, self).__init__()

        # Convolutional and pooling layers
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5)  # Output: (16, 169, 82)
        self.pool1 = nn.MaxPool2d(2, 2)              # Output: (16, 84, 41)

        self.conv2 = nn.Conv2d(16, 32, kernel_size=5) # Output: (32, 80, 37)
        self.pool2 = nn.MaxPool2d(2, 2)              # Output: (32, 40, 18)

        # Dropout after convolutional layers
        self.dropout_conv = nn.Dropout2d(p=dropout_conv_p)  # Dropout2d for 2D data

        # Fully connected layers
        self.fc1 = nn.Linear(32 * 40 * 18, 256)      # Adjusted input size
        self.fc2 = nn.Linear(256, 128)              # Fully connected layer 2
        self.fc3 = nn.Linear(128, 1)              

        # Dropout for fully connected layers
        self.dropout_fc = nn.Dropout(p=dropout_fc_p)

    def forward(self, x):
        # Convolutional layers
        x = self.conv1(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.pool1(x)  # Pooling
        x = self.dropout_conv(x)  # Dropout (no effect if p=0.0)

        x = self.conv2(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.pool2(x)  # Pooling
        x = self.dropout_conv(x)  # Dropout (no effect if p=0.0)

        # Flatten
        x = torch.flatten(x, 1)

        # Fully connected layers
        x = self.fc1(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.dropout_fc(x)  # Dropout (no effect if p=0.0)

        x = self.fc2(x)
        x = F.relu(x)  # Apply ReLU activation
        x = self.dropout_fc(x)  # Dropout (no effect if p=0.0)
        x = torch.sigmoid(self.fc3(x))  # Output layer
        
        return x

In [4]:
from sklearn.metrics import f1_score

def train(model, criterion, optimizer, num_epochs, train_loader, val_loader, model_name = "cnn"):
    model.to(device)
    
    PRINT_STEP = len(train_loader) // 5 - 1
    epochs_without_val_acc_improvement = 0
    best_val_acc = 0.0

    for epoch in range(num_epochs):
        model.train()

        running_loss = 0.0
        correct = 0
        all_labels = []
        all_preds = []

        for  i, data in enumerate(train_loader):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.unsqueeze(1)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            preds = (outputs > 0.5).float()  # Binary prediction with threshold 0.5
            all_labels.extend(labels.cpu().numpy().flatten())
            all_preds.extend(preds.detach().cpu().numpy().flatten())

            correct += (preds == labels).float().mean().item()
            
            running_loss += loss.item()

            if i % PRINT_STEP == PRINT_STEP-1:
                accuracy = correct / PRINT_STEP
                loss = running_loss / PRINT_STEP
                step = epoch * len(train_loader) + i
                wandb.log({
                        "train/accuracy": accuracy,
                        "train/loss": loss
                    },
                    step=step
                )
                running_loss = 0.0
                correct = 0

        f1 = f1_score(all_labels, all_preds, average='macro')
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}, F1 Score: {f1:.4f}")

        # Validation
        model.eval()

        val_loss = 0.0
        val_correct = 0

        with torch.no_grad():
            for j, data in enumerate(val_loader):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)
                labels = labels.unsqueeze(1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                # Binary predictions
                preds = (outputs > 0.5).float()
                val_correct += (preds == labels).float().mean().item()

        accuracy = val_correct / len(val_loader)
        loss = val_loss / len(val_loader)
        wandb.log({
                "validation/accuracy": accuracy,
                "validation/loss": loss
            },
            step=(epoch + 1) * len(train_loader)
        )

        if accuracy > best_val_acc:
            best_val_acc = accuracy
            epochs_without_val_acc_improvement = 0
        else:
            epochs_without_val_acc_improvement += 1
        if epochs_without_val_acc_improvement >= 10:
            print("8 epochs without a val accuracy improvement. Stopping the train")
            break
        if epoch > 15 and accuracy < 0.9:
            print("Too low accuracy. Stopping the train")
            break

    print("Training complete.")
    
def test(model, test_loader):
    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on test images: {100 * correct // total} %')

## Data

In [5]:
from utils.dataset_creator import SpecgramsRandomFilter, SpecgramsSilentFilter

DATASETS_PARENT_PATH = 'datasets'
DATASET_PATH = 'dataset'
DATA_DIR = os.path.join('../../', DATASETS_PARENT_PATH, DATASET_PATH)

In [6]:
from prepare_datasets import create_datasets

create_datasets(DATASETS_PARENT_PATH, DATASET_PATH, [
    SpecgramsSilentFilter(1.1)
])

DatasetCreator: Exporting the dataset with the following parameters:
    n_fft=1024
    n_mels=86
    duration_seconds=2s
    multithreading=True
Class 0 recordings count: 270
Class 1 recordings count: 630
Image properties:
    width=173px
    height=86px
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/train/0
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/train/1
Finished [900/900]Dataset has been exported. Elapsed time: 139.80219292640686s.
DatasetCreator: Exporting the dataset with the following parameters:
    n_fft=1024
    n_mels=86
    duration_seconds=2s
    multithreading=True
Class 0 recordings count: 90
Class 1 recordings count: 210
Image properties:
    width=173px
    height=86px
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/validation/0
Removing directory /home/migoox/dev/ml-dogs-sharepoint/src/../datasets/dataset/validation/1
Finished [300/300]Dataset has been exported. Elapse

In [7]:
INPUT_WIDTH = 67
INPUT_HEIGHT = 66

LEARNING_RATE = 0.001
LR_DECAY = 0.95
BATCH_SIZE = 64
EPOCHS = 30

LEARNING_RATE_ARRAY = [0.1, 0.01, 0.001, 0.0001]
BATCH_SIZE_ARRAY = [32, 64, 128]
DROPOUT_ARRAY = [0, 0.2, 0.4]


## Training

In [8]:
for lr, b, d1, d2 in itertools.product(LEARNING_RATE_ARRAY, BATCH_SIZE_ARRAY, DROPOUT_ARRAY, DROPOUT_ARRAY):
    print(f"lr:{lr}-b:{b}-d_c:{d1}, d_f:{d2}")
    wandb.init(
        project="iml_final_tests",
        config={
            "learning_rate": lr,
            "learning_rate_decay": LR_DECAY,
            "batch_size": b,
            "input_resolution": (INPUT_WIDTH, INPUT_HEIGHT),
            "architecture": "CNN",
            "dataset": "DAPS",
            "fixed": "true"
        },
        name=f"lr:{lr}-b:{b}-d_c:{d1}, d_f:{d2}"
    )
    model = Net()
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    transform = transforms.ToTensor()
    
    train_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "train"), transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    val_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "validation"), transform=transform)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    test_dataset = SpectrogramDataset(data_dir=os.path.join(DATA_DIR, "test"),transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    train(model, criterion, optimizer, EPOCHS, train_loader, val_loader, model_name="simple_cnn")

    wandb.finish()


lr:0.1-b:32-d_c:0, d_f:0


Epoch 1/30, Loss: 1.9706, F1 Score: 0.4095
Epoch 2/30, Loss: 2.2797, F1 Score: 0.3996
Epoch 3/30, Loss: 2.1252, F1 Score: 0.3996
Epoch 4/30, Loss: 2.0895, F1 Score: 0.3996
Epoch 5/30, Loss: 1.6285, F1 Score: 0.3996
Epoch 6/30, Loss: 2.2440, F1 Score: 0.3996
Epoch 7/30, Loss: 1.9706, F1 Score: 0.3996
Epoch 8/30, Loss: 2.2599, F1 Score: 0.3996
Epoch 9/30, Loss: 2.3642, F1 Score: 0.3996
Epoch 10/30, Loss: 2.0525, F1 Score: 0.3996
Epoch 11/30, Loss: 2.1925, F1 Score: 0.3996
8 epochs without a val accuracy improvement. Stopping the train
Training complete.


train/accuracy,▄▄▄▅▃▆▆▇▄▃▄▂▄▆▄▄▃▅▁▃▅▇▅▄▄▅▄▅▄▃▆▅▇▃█▄▂▆▆▃
train/loss,▁▅▆▄▆▃▃▂▆▁▅▇▅▄▃▆▂▅▆▅▆▇▃▃▄▅▄▅▄▅▅▃▄▃▇▅█▃▃▆
validation/accuracy,▁▁▁▁▁▁▁▁▁▁▁
validation/loss,▁▁▁▁▁▁▁▁▁▁▁
train/accuracy,0.65625
train/loss,34.375
validation/accuracy,0.66927
validation/loss,33.07292


lr:0.1-b:32-d_c:0, d_f:0.2


Epoch 1/30, Loss: 2.2242, F1 Score: 0.4083
Epoch 2/30, Loss: 2.5174, F1 Score: 0.3996
Epoch 3/30, Loss: 2.3603, F1 Score: 0.3996
Epoch 4/30, Loss: 2.2255, F1 Score: 0.3996
Epoch 5/30, Loss: 1.9878, F1 Score: 0.3996
Epoch 6/30, Loss: 2.1397, F1 Score: 0.3996
Epoch 7/30, Loss: 2.2070, F1 Score: 0.3996
Epoch 8/30, Loss: 2.1912, F1 Score: 0.3996
Epoch 9/30, Loss: 2.4289, F1 Score: 0.3996
Epoch 10/30, Loss: 1.9019, F1 Score: 0.3996
Epoch 11/30, Loss: 2.0908, F1 Score: 0.3996
8 epochs without a val accuracy improvement. Stopping the train
Training complete.


...

Epoch 1/30, Loss: 0.0222, F1 Score: 0.6359
Epoch 2/30, Loss: 0.0135, F1 Score: 0.8729
Epoch 3/30, Loss: 0.0065, F1 Score: 0.9417
Epoch 4/30, Loss: 0.0042, F1 Score: 0.9724
Epoch 5/30, Loss: 0.0019, F1 Score: 0.9850
Epoch 6/30, Loss: 0.0010, F1 Score: 0.9893
Epoch 7/30, Loss: 0.0009, F1 Score: 0.9936
Epoch 8/30, Loss: 0.0008, F1 Score: 0.9963
Epoch 9/30, Loss: 0.0001, F1 Score: 0.9990
Epoch 10/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 11/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 12/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 13/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 14/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 15/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 16/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 17/30, Loss: 0.0000, F1 Score: 1.0000
Too low accuracy. Stopping the train
Training complete.


train/accuracy,▁▁▂▃▄▇▇▇▇▇██████████████████████████████
train/loss,██▇▇▆▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation/accuracy,▁▅▇▅▇█▇███▇██████
validation/loss,▂▁▁▂▂▂▃▃▅▅▇▇▇████
train/accuracy,1
train/loss,6e-05
validation/accuracy,0.8945
validation/loss,1.42713


lr:0.001-b:128-d_c:0.2, d_f:0.2


Epoch 1/30, Loss: 0.0272, F1 Score: 0.5552
Epoch 2/30, Loss: 0.0127, F1 Score: 0.8661
Epoch 3/30, Loss: 0.0056, F1 Score: 0.9410
Epoch 4/30, Loss: 0.0021, F1 Score: 0.9728
Epoch 5/30, Loss: 0.0044, F1 Score: 0.9803
Epoch 6/30, Loss: 0.0017, F1 Score: 0.9897
Epoch 7/30, Loss: 0.0037, F1 Score: 0.9924
Epoch 8/30, Loss: 0.0027, F1 Score: 0.9940
Epoch 9/30, Loss: 0.0006, F1 Score: 0.9856
Epoch 10/30, Loss: 0.0001, F1 Score: 0.9963
Epoch 11/30, Loss: 0.0000, F1 Score: 0.9992
Epoch 12/30, Loss: 0.0000, F1 Score: 0.9998
Epoch 13/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 14/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 15/30, Loss: 0.0000, F1 Score: 1.0000
Epoch 16/30, Loss: 0.0000, F1 Score: 1.0000


KeyboardInterrupt: 

In [17]:
wandb.finish()

train/accuracy,▁
train/loss,▁
train/accuracy,0.70072
train/loss,0.58681
